In [1]:
%load_ext autoreload
%autoreload 2

In [28]:
import os, sys
import time
import pickle

import numpy as np
import pandas as pd

import rdkit
import rdkit.Chem as chem
import rdkit.Chem.AllChem as allchem

from gryffin import Gryffin

from molar_interface import MolarInterface

## Make connection to the MolarDB 

In [29]:
# define user details
user_details = {
    'email':'riley.hickman13@gmail.com',
    'password':'az1$jH5v',
}


In [30]:
db = MolarInterface(user_details, database_name='madness_laser') # client in client attribute
db.client.test_token()

{'email': 'riley.hickman13@gmail.com',
 'is_superuser': False,
 'is_active': True,
 'full_name': 'rileyhickman',
 'created_on': '2022-01-18T19:16:33.241173',
 'user_id': 3}

## Retrieve the first set of molecules in the DB

We use a filtered query to return all of the molecules for which we have tried to measure the optical properties. 

In [5]:
# custom query to get the molecules with the optical properties
df = db.client.query_database(
    [
      'machine',
      'lab',
      'synthesis',
      'molecule',
      'molecule_molecule',
      'molecule.optical_properties',
    ],
    joins = [
      {
        'type': 'molecule',
        'on': {
          'column1': 'molecule.molecule_id',
          'column2': 'synthesis.molecule_id',
            
        },
      },
      {
        'type': 'molecule_molecule',
        'on': {
          'column1': 'molecule_molecule.molecule_id',
          'column2': 'molecule.molecule_id',
        },
      },
      {
        'type': 'machine',
        'on': {
          'column1': 'machine.machine_id',
          'column2': 'synthesis.machine_id',
        },
      },
      {
        'type': 'lab',
        'on': {
          'column1': 'lab.lab_id',
          'column2': 'synthesis.lab_id',
        },
      },
    ],
limit=1000)

In [6]:
print(df.shape)
df.head()

(187, 33)


,machine.machine_id,machine.created_on,machine.updated_on,machine.name,machine.lab_id,lab.lab_id,lab.created_on,lab.updated_on,lab.name,synthesis.synthesis_id,...,molecule_molecule.created_on,molecule_molecule.updated_on,molecule_molecule.fragment_a,molecule_molecule.fragment_b,molecule_molecule.fragment_c,molecule_molecule.molecule_id,synthesis.parcel_tracking_num,molecule.hid,molecule.commercially_available,molecule.molecule_type_id
0,5600f641-b180-41b1-8043-921e66107744,2022-01-18T12:59:47.995614,2022-01-18T12:59:47.995614,The Machine,b65835e3-b908-45c6-9995-ec98d0caafe7,b65835e3-b908-45c6-9995-ec98d0caafe7,2022-01-18T12:59:47.145756,2022-01-18T12:59:47.145756,Illinois,8cf4e82b-6075-460d-a2ad-cb492f4c4b93,...,2022-02-03T14:24:29.817992,2022-02-03T14:24:29.817992,56471498-825e-47e7-a772-136f0f7bd817,d1bb232c-b83a-49cb-bcec-0e02558a6c41,fb544fb8-5588-41af-ab92-fcc8c3b4e052,b33b1cc9-81d8-4833-803f-466411581a4d,None,None,None,None
1,5600f641-b180-41b1-8043-921e66107744,2022-01-18T12:59:47.995614,2022-01-18T12:59:47.995614,The Machine,b65835e3-b908-45c6-9995-ec98d0caafe7,b65835e3-b908-45c6-9995-ec98d0caafe7,2022-01-18T12:59:47.145756,2022-01-18T12:59:47.145756,Illinois,43283ee2-6110-4460-aa57-08cedd716cbd,...,2022-02-09T22:03:57.061626,2022-02-09T22:03:57.061626,05ca4304-bea5-41b7-a7f1-c2bc46462153,d1bb232c-b83a-49cb-bcec-0e02558a6c41,b0ee7203-c40b-4f5b-9b32-c647bd61ddf1,322b8304-39b5-49bc-a307-99719f0d5911,775853526901,None,None,None
2,f158c26e-a94e-4994-9a95-de324aa1da23,2022-01-18T12:59:46.312079,2022-01-18T12:59:46.312079,ChemSpeed,08cc75ca-8d1e-4ead-9b56-ad1b6ba83a41,08cc75ca-8d1e-4ead-9b56-ad1b6ba83a41,2022-01-18T12:59:44.418607,2022-01-18T12:59:44.418607,Toronto,695c1c55-bf6a-4265-b68d-4655999a7b85,...,2022-02-11T16:19:41.240088,2022-02-11T16:19:41.240088,6bf2b5a1-c157-4b5a-8a80-bf0b44704273,45cb409f-376c-4f89-8ee7-79595d2668e7,2d83fadb-4b36-42a3-a29d-1ff30b8972d6,539c5e5e-25fa-457d-a2c0-ecf3f88c36f3,None,A001B003C001,False,a06a38b6-78b6-4fcc-8a61-3fe9d0af7cea
3,f158c26e-a94e-4994-9a95-de324aa1da23,2022-01-18T12:59:46.312079,2022-01-18T12:59:46.312079,ChemSpeed,08cc75ca-8d1e-4ead-9b56-ad1b6ba83a41,08cc75ca-8d1e-4ead-9b56-ad1b6ba83a41,2022-01-18T12:59:44.418607,2022-01-18T12:59:44.418607,Toronto,9a71f775-c341-42b1-ab7a-cd0917b5e80f,...,2022-02-11T16:19:42.453093,2022-02-11T16:19:42.453093,6bf2b5a1-c157-4b5a-8a80-bf0b44704273,45cb409f-376c-4f89-8ee7-79595d2668e7,d84002d6-88ab-4007-80e3-629404e64478,3275cec5-919c-4f18-bb4c-2ec941f7db3b,None,A001B003C002,False,a06a38b6-78b6-4fcc-8a61-3fe9d0af7cea
4,f158c26e-a94e-4994-9a95-de324aa1da23,2022-01-18T12:59:46.312079,2022-01-18T12:59:46.312079,ChemSpeed,08cc75ca-8d1e-4ead-9b56-ad1b6ba83a41,08cc75ca-8d1e-4ead-9b56-ad1b6ba83a41,2022-01-18T12:59:44.418607,2022-01-18T12:59:44.418607,Toronto,8cf70e33-a879-4ab7-879f-5284da51c300,...,2022-02-11T16:19:43.720683,2022-02-11T16:19:43.720683,6bf2b5a1-c157-4b5a-8a80-bf0b44704273,45cb409f-376c-4f89-8ee7-79595d2668e7,c995b067-f927-40cc-98ce-f067b07ef086,63bd1d6d-3e81-48ae-9535-e04dc6d9d32f,None,A001B003C003,False,a06a38b6-78b6-4fcc-8a61-3fe9d0af7cea


In [7]:
df['synthesis.status'].unique()

array(['PROCESSING', 'RECEIVED', 'DONE', 'FAILED', 'SYNTHESIZED',
       'ACQUIRED'], dtype=object)

In [8]:
# get the sub dataframe of all molecules that are either SHIPPED OR IN PROCESSING
# these will be constrained in the Gryffin acquisition
df_proc = df[df['synthesis.status'].isin(['SHIPPED', 'PROCESSING'])]
print('NUM IN PROGRESS : ', df_proc.shape[0])

# get all the entries for which the status is DONE
df_done = df[df['synthesis.status'].isin(['DONE', 'FAILED'])]
print('NUM FINISHED : ', df_done.shape[0])

print(f'PERCENT DONE {(df_done.shape[0] / (33*30*165) *100):.3f}')  # TODO: Make space calculaiton automatic from DB?

df_proc

NUM IN PROGRESS :  2
NUM FINISHED :  173
PERCENT DONE 0.106


,machine.machine_id,machine.created_on,machine.updated_on,machine.name,machine.lab_id,lab.lab_id,lab.created_on,lab.updated_on,lab.name,synthesis.synthesis_id,...,molecule_molecule.created_on,molecule_molecule.updated_on,molecule_molecule.fragment_a,molecule_molecule.fragment_b,molecule_molecule.fragment_c,molecule_molecule.molecule_id,synthesis.parcel_tracking_num,molecule.hid,molecule.commercially_available,molecule.molecule_type_id
0,5600f641-b180-41b1-8043-921e66107744,2022-01-18T12:59:47.995614,2022-01-18T12:59:47.995614,The Machine,b65835e3-b908-45c6-9995-ec98d0caafe7,b65835e3-b908-45c6-9995-ec98d0caafe7,2022-01-18T12:59:47.145756,2022-01-18T12:59:47.145756,Illinois,8cf4e82b-6075-460d-a2ad-cb492f4c4b93,...,2022-02-03T14:24:29.817992,2022-02-03T14:24:29.817992,56471498-825e-47e7-a772-136f0f7bd817,d1bb232c-b83a-49cb-bcec-0e02558a6c41,fb544fb8-5588-41af-ab92-fcc8c3b4e052,b33b1cc9-81d8-4833-803f-466411581a4d,None,None,None,None
176,9db0b589-ae14-44bb-911f-457b0cd3b0d7,2022-01-18T12:59:49.708708,2022-01-18T12:59:49.708708,Chemputer,6678a008-deb4-4608-9cb2-d32981156538,6678a008-deb4-4608-9cb2-d32981156538,2022-01-18T12:59:48.810040,2022-01-18T12:59:48.810040,Glasgow,e1980582-3f9b-4b0c-832f-e238e11bccf5,...,2022-02-24T16:19:25.646748,2022-02-24T16:19:25.646748,8b25a7f2-4195-4ac9-889a-fb4638832aa9,None,None,fbb2014b-318e-4443-b764-8871663b6165,None,None,None,None


In [9]:
# ATTN: Dropping 176 because of issues with fragment IDs (and attributed to Chemputer...)
df_proc.drop(176, inplace=True)
df_proc

,machine.machine_id,machine.created_on,machine.updated_on,machine.name,machine.lab_id,lab.lab_id,lab.created_on,lab.updated_on,lab.name,synthesis.synthesis_id,...,molecule_molecule.created_on,molecule_molecule.updated_on,molecule_molecule.fragment_a,molecule_molecule.fragment_b,molecule_molecule.fragment_c,molecule_molecule.molecule_id,synthesis.parcel_tracking_num,molecule.hid,molecule.commercially_available,molecule.molecule_type_id
0,5600f641-b180-41b1-8043-921e66107744,2022-01-18T12:59:47.995614,2022-01-18T12:59:47.995614,The Machine,b65835e3-b908-45c6-9995-ec98d0caafe7,b65835e3-b908-45c6-9995-ec98d0caafe7,2022-01-18T12:59:47.145756,2022-01-18T12:59:47.145756,Illinois,8cf4e82b-6075-460d-a2ad-cb492f4c4b93,...,2022-02-03T14:24:29.817992,2022-02-03T14:24:29.817992,56471498-825e-47e7-a772-136f0f7bd817,d1bb232c-b83a-49cb-bcec-0e02558a6c41,fb544fb8-5588-41af-ab92-fcc8c3b4e052,b33b1cc9-81d8-4833-803f-466411581a4d,None,None,None,None


In [10]:
# get the fragment details for the fragments

def make_gryffin_observations(df_done):
    ''' prepare typical list of dictionary-style observations for Gryffin. This function
    considers all failed experiments as nan-valued objectives. We do not consider the cause
    for failure here (future improvement). This function is quite slow with all the calls to 
    the db about the fragments - make more efficient
    '''
    # prepare observation list for Gryffin
    observations = []

    for _, row in df_done.iterrows():
        row = row.to_dict()
        frag_a_hid = db.get_fragment_details(
                    row['molecule_molecule.fragment_a'], identifier_type='molecule_id'
            )['id']
        frag_b_hid = db.get_fragment_details(
                    row['molecule_molecule.fragment_b'], identifier_type='molecule_id'
            )['id']
        frag_c_hid = db.get_fragment_details(
                    row['molecule_molecule.fragment_c'], identifier_type='molecule_id'
            )['id']
        opt_prop = row['molecule.optical_properties']

        if row['synthesis.status']=='DONE':
            # we should have some optical properties, we can use this data to train Gryffin
            assert opt_prop!={}
            obj = opt_prop['gain_cross_section']
            assert type(obj)==float

        elif row['synthesis.status']=='FAILED':
            # failed experiment, pass value to gryffin with nan objective value
            obj = np.nan
        else:
            raise NotImplementedError
        observations.append(
            {'frag_a': frag_a_hid, 'frag_b': frag_b_hid, 'frag_c': frag_c_hid, 'obj': obj}
        )
        
    return observations


    
def get_processing_molecules(df_proc):
    ''' prepare a list of molecule hids which are in progress
    '''
    proc_hids = []
    for _, row in df_proc.iterrows():
        row = row.to_dict()
        frag_a_hid = db.get_fragment_details(
                    row['molecule_molecule.fragment_a'], identifier_type='molecule_id'
            )['id']
        frag_b_hid = db.get_fragment_details(
                    row['molecule_molecule.fragment_b'], identifier_type='molecule_id'
            )['id']
        frag_c_hid = db.get_fragment_details(
                    row['molecule_molecule.fragment_c'], identifier_type='molecule_id'
            )['id']
        mol_hid = ''.join([frag_a_hid, frag_b_hid, frag_c_hid])
        proc_hids.append(mol_hid)
    return proc_hids
    
def known_constraints(param):
    ''' known constraint that the suggested molecule must not be either shipping or
    in progress. Avoids redundant suggestions
    '''
    # check if the param is within the known molecules
    query_hid = ''.join([param['frag_a'], param['frag_b'], param['frag_c']])
    crit1 = query_hid in proc_hids
    
    # check to see if the sample can be made in at least one single location
    crit2 = False
    # TODO: this in only for the first batch, constrain to Toronto ONLY!
    for lab_frags in [toronto_frags, uiuc_frags]:
        if (
            (param['frag_a'] in lab_frags['frag_a']) & (param['frag_b'] in lab_frags['frag_b']) & (param['frag_c'] in lab_frags['frag_c'])):
            crit2=True
            
    return (not(crit1))&crit2


In [11]:
# get hids of molecules in process
proc_hids = get_processing_molecules(df_proc)
proc_hids

['A012B025C130']

In [12]:
# get gryffin observations
observations = make_gryffin_observations(df_done)

In [13]:
# get all the unique frags in the previous observations to add onto the available 
# fragments (otherwise Gryffin will crash)
a_prev_hids = list(set([o['frag_a'] for o in observations]))
b_prev_hids = list(set([o['frag_b'] for o in observations]))
c_prev_hids = list(set([o['frag_c'] for o in observations]))

In [14]:
def get_lab_frags(lab):
    ''' returns the set of available fragments for a specific laboratory
    '''
    frags=db.get_lab_avail_frags(lab=lab)
    a_hids = frags[frags['molecule.hid'].str.contains('A')]['molecule.hid'].tolist()
    b_hids = frags[frags['molecule.hid'].str.contains('B')]['molecule.hid'].tolist()
    c_hids = frags[frags['molecule.hid'].str.contains('C')]['molecule.hid'].tolist()
    
    return {'frag_a': a_hids, 'frag_b': b_hids, 'frag_c': c_hids}

def get_frags(strategy='set'):
    ''' return all the unique molecules across all labs
    '''
    a_hids_all = []
    b_hids_all = []
    c_hids_all = []
    
    for lab in ['Toronto', 'Illinois']:
        lab_frags = get_lab_frags(lab)
        a_hids_all.extend(lab_frags['frag_a'])
        b_hids_all.extend(lab_frags['frag_b'])
        c_hids_all.extend(lab_frags['frag_c'])
    
    a_hids_all = list(set(a_hids_all))
    b_hids_all = list(set(b_hids_all))
    c_hids_all = list(set(c_hids_all))
    
    return a_hids_all, b_hids_all, c_hids_all
    

In [15]:

a_hids_all, b_hids_all, c_hids_all = get_frags()

# add fragments from previous iterations
a_hids_all = list(set(a_hids_all + a_prev_hids))
b_hids_all = list(set(b_hids_all + b_prev_hids))
c_hids_all = list(set(c_hids_all + c_prev_hids))

print('NUM A FRAGS : ', len(a_hids_all))
print('NUM B FRAGS : ', len(b_hids_all))
print('NUM C FRAGS : ', len(c_hids_all))

print('NUM ACCESSIBLE MOLS : ', len(a_hids_all)*len(b_hids_all)*len(c_hids_all))

NUM A FRAGS :  33
NUM B FRAGS :  55
NUM C FRAGS :  161
NUM ACCESSIBLE MOLS :  292215


In [16]:
# get toronto and UIUC available frags
toronto_frags = get_lab_frags('Toronto')
uiuc_frags = get_lab_frags('Illinois')

## Build Gryffin 

In [17]:
def get_descriptors(hid, descriptors):
    ''' looks up and returns descriptors for a particular fragment
    
    Args:
        hid (str): human readable identifier of the fragment
        descriptors (pd.DataFrame): dataframe corresponding to the type of fragment
    '''
    return descriptors[descriptors.hid==hid].iloc[0, 1:].values.astype(np.float).tolist()

def mol_hid_from_params(params):
    mol_hids = []
    for param in params:
        mol_hids.append(''.join([param['frag_a'], param['frag_b'], param['frag_c']]))
    
    return mol_hids
        

In [18]:
# load descriptors from disk
pca_desc_a = pd.read_csv('descriptors/gen2_pca_desc_a.csv', index_col=None)
pca_desc_b = pd.read_csv('descriptors/gen2_pca_desc_b.csv', index_col=None)
pca_desc_c = pd.read_csv('descriptors/gen2_pca_desc_c.csv', index_col=None)
print('NUM DESC A ', pca_desc_a.shape[1]-1)
print('NUM DESC B ', pca_desc_b.shape[1]-1)
print('NUM DESC C ', pca_desc_c.shape[1]-1)

NUM DESC A  30
NUM DESC B  30
NUM DESC C  35


In [19]:
# set the available fragments to make the Gryffin search space 
# TODO: change this to only those available
avail_frags_a = a_hids_all # pca_desc_a['hid'].tolist()
avail_frags_b = b_hids_all # pca_desc_b['hid'].tolist()
avail_frags_c = c_hids_all # pca_desc_c['hid'].tolist()
print('NUM AVAIL FRAGS A : ', len(avail_frags_a))
print('NUM AVAIL FRAGS B : ', len(avail_frags_b))
print('NUM AVAIL FRAGS C : ', len(avail_frags_c))

NUM AVAIL FRAGS A :  33
NUM AVAIL FRAGS B :  55
NUM AVAIL FRAGS C :  161


In [20]:
# Gryffin config

#sampling_strategies = np.linspace(-1, 1, 40)
sampling_strategies_1 = np.linspace(0.6, 1, 30)
sampling_strategies_2 = np.linspace(-1, 0.5, 10)
sampling_strategies = np.concatenate((sampling_strategies_1, sampling_strategies_2))
#sampling_strategies = np.linspace(-1, 0.45, 20)

# generate descritptors
# FRAG_A_DESC = {i: get_descriptors(i, pca_desc_a) for i in avail_frags_a}
# FRAG_B_DESC = {i: get_descriptors(i, pca_desc_b) for i in avail_frags_b}
# FRAG_C_DESC = {i: get_descriptors(i, pca_desc_c) for i in avail_frags_c}

FRAG_A_DESC = {i: None for i in avail_frags_a}
FRAG_B_DESC = {i: None for i in avail_frags_b}
FRAG_C_DESC = {i: None for i in avail_frags_c}

# fwa = feasibility-weighted acquisition
# fia = feasibility-interpolated acquisition
# fca = feasibility-constrained acquisition --> use, param=0.2 ish works best on cat benchmarks

config = {
     "general": {
             "backend": 'tensorflow',
             "num_cpus": 4,
             "auto_desc_gen": False, # dynamic Gryffin
             "batches": 1,
             "sampling_strategies": 1,
             "feas_approach": 'fca',
             "feas_param": 0.2,
             "boosted":  True,
             "caching": False,
             "random_seed": 22031996,
             "acquisition_optimizer": 'genetic',
             "verbosity": 3
                },
    "parameters": [
        {"name": "frag_a", "type": "categorical", "category_details": FRAG_A_DESC },
        {"name": "frag_b", "type": "categorical", "category_details": FRAG_B_DESC },
        {"name": "frag_c", "type": "categorical", "category_details": FRAG_C_DESC },
      
    ],
    "objectives": [
        {"name": "obj", "goal": "max"},
    ]
}

In [21]:
# instantiate gryffin
gryffin = Gryffin(config_dict=config, known_constraints=known_constraints)

In [22]:
# recommend a batch of samples
samples = []
for ix, sampling_strat in enumerate(sampling_strategies):
    
    sample = gryffin.recommend(observations, sampling_strategies=[sampling_strat])
    print(f'\nSAMPLING STRATEGY : {sampling_strat} SAMPLE : {sample}\n')
    samples.extend(sample) 
    

173 observations found

2022-08-14 18:14:17.584007: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-14 18:14:17.856909: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-14 18:14:17.857562: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda/lib64
2022-08-14 18:14:17.857615: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11:

Bayesian neural network trained in 48.4 s

1200 random proposals drawn in 31.3 s

1200 proposals optimized in 7.7 s using 4 CPUs

...

Acquisition tasks for 1 sampling strategy performed in 40.5 s

1 sample selected in 1.3 s

Memory usage: 4 GB, 4247 MB, 992 kB

Overall time required: 1 min 29 s


SAMPLING STRATEGY : 0.5 SAMPLE : [{'frag_a': 'A001', 'frag_b': 'B036', 'frag_c': 'C141'}]



In [23]:
def check_duplicates(samples, observations):
    ''' returns True if there are duplicates, False if there are none
    '''
    sample_hids = [''.join([s['frag_a'],s['frag_b'],s['frag_c']]) for s in samples]
    observ_hids = [''.join([o['frag_a'],o['frag_b'],o['frag_c']]) for o in observations]
    
    # check duplicates over previous observations 
    is_obs_dup = any(s in observ_hids for s in sample_hids)
    
    # check duplicates within the batch
    is_samp_dup = len(set(sample_hids))<len(sample_hids)
    
    return is_obs_dup, is_samp_dup

In [24]:
check_duplicates(samples, observations)

(False, False)

In [25]:
samples

[{'frag_a': 'A001', 'frag_b': 'B046', 'frag_c': 'C003'},
 {'frag_a': 'A041', 'frag_b': 'B058', 'frag_c': 'C100'},
 {'frag_a': 'A001', 'frag_b': 'B031', 'frag_c': 'C021'},
 {'frag_a': 'A028', 'frag_b': 'B038', 'frag_c': 'C085'},
 {'frag_a': 'A039', 'frag_b': 'B060', 'frag_c': 'C079'},
 {'frag_a': 'A003', 'frag_b': 'B055', 'frag_c': 'C098'},
 {'frag_a': 'A042', 'frag_b': 'B039', 'frag_c': 'C018'},
 {'frag_a': 'A001', 'frag_b': 'B052', 'frag_c': 'C015'},
 {'frag_a': 'A001', 'frag_b': 'B054', 'frag_c': 'C013'},
 {'frag_a': 'A001', 'frag_b': 'B052', 'frag_c': 'C011'},
 {'frag_a': 'A001', 'frag_b': 'B051', 'frag_c': 'C090'},
 {'frag_a': 'A001', 'frag_b': 'B037', 'frag_c': 'C022'},
 {'frag_a': 'A001', 'frag_b': 'B059', 'frag_c': 'C087'},
 {'frag_a': 'A001', 'frag_b': 'B048', 'frag_c': 'C003'},
 {'frag_a': 'A015', 'frag_b': 'B062', 'frag_c': 'C018'},
 {'frag_a': 'A004', 'frag_b': 'B051', 'frag_c': 'C007'},
 {'frag_a': 'A001', 'frag_b': 'B031', 'frag_c': 'C020'},
 {'frag_a': 'A001', 'frag_b': '

In [31]:
rxn_transmut = allchem.ReactionFromSmarts("[*:1][Br,I]>>[*:1][Po]")
rxn_b_c = allchem.ReactionFromSmarts(
    "[Br,I][*:1].C[N+]12CC(=O)O[B-]1([*:2])OC(=O)C2>>[*:1]-[*:2]"
)
rxn_a_b_c = allchem.ReactionFromSmarts("CC1(C)OB([*:1])OC1(C)C.[*:2][Po]>>[*:1]-[*:2]")
rxn_a_b_c2 = allchem.ReactionFromSmarts("[*:1]B(O)O.[*:2][Po]>>[*:1]-[*:2]")


def assemble_fragments(hid_a, hid_b, hid_c):
    smiles_a = db.get_fragment_details(hid_a, identifier_type='hid')['smiles']
    smiles_b = db.get_fragment_details(hid_b, identifier_type='hid')['smiles']
    smiles_c = db.get_fragment_details(hid_c, identifier_type='hid')['smiles']

    a = chem.MolFromSmiles(smiles_a)
    b = chem.MolFromSmiles(smiles_b)
    c = chem.MolFromSmiles(smiles_c)
    if a is None or b is None or c is None:
        raise ValueError("Invalid SMILES")

    # Building the molecule... there is probably a better way
    b_transmut = rxn_transmut.RunReactants([b])[0][0]
    b_c = rxn_b_c.RunReactants([c, b_transmut])[0][0]
    b_c_b = rxn_b_c.RunReactants([b_c, b_transmut])[0][0]
    try:
        a_b_c_b = rxn_a_b_c.RunReactants([a, b_c_b])[0][0]
        a_b_c_b_a = rxn_a_b_c.RunReactants([a, a_b_c_b])[0][0]
    except:
        a_b_c_b = rxn_a_b_c2.RunReactants([a, b_c_b])[0][0]
        a_b_c_b_a = rxn_a_b_c2.RunReactants([a, a_b_c_b])[0][0]
    return chem.MolToSmiles(a_b_c_b_a)

In [32]:
molecule_smiles = []
molecule_hids = []

for sample in samples:
    molecule_hids.append(''.join([sample['frag_a'],sample['frag_b'],sample['frag_c']]))
    molecule_smiles.append(assemble_fragments(sample['frag_a'], sample['frag_b'], sample['frag_c']))
    

In [33]:
molecule_mols = [chem.MolFromSmiles(s) for s in molecule_smiles]


In [34]:
img = chem.Draw.MolsToGridImage(
    molecule_mols, molsPerRow=5, subImgSize=(400,400), returnPNG=False, legends=molecule_hids,
)
img.save('round_3_samples_all.png')

In [35]:
with open('round_3_samples_all.txt', 'w') as f:
    for smile, hid in zip(molecule_smiles, molecule_hids):
        f.write(f'{smile}\t{hid}\n')


## Write the new target molecules/syntheses to the DB

In [37]:
for sample, smi in zip(samples, molecule_smiles):
    print(f'>> Creating DB entry for sample {sample["frag_a"]} {sample["frag_b"]} {sample["frag_c"]}...')
    db.create_target_compound(
            smiles=smi,
            fragment_a=sample['frag_a'],
            fragment_b=sample['frag_b'],
            fragment_c=sample['frag_c'],
            fragment_identifier='hid',
    )

>> Creating DB entry for sample A001 B046 C003...


MolarBackendError: The client got an unexpected answer from the backend:
401: Unique constraint violation!

In [36]:
# db.get_fragment_details('A011', identifier_type='hid')

{'id': 'A011',
 'smiles': 'OB(O)c1ccc2c3ccccc3n(-c3ccccc3)c2c1',
 'CAS': '1001911-63-2',
 'molecule_id': 'b230ce95-ad00-4920-9cd3-5d344ea521b6'}

In [37]:
# db.client.query_database('synthesis', limit=1000)

,synthesis_id,created_on,updated_on,status,lab_id,molecule_id,machine_id,parcel_tracking_num
0,8cf4e82b-6075-460d-a2ad-cb492f4c4b93,2022-02-03T14:24:29.904436,2022-02-03T14:24:29.904436,PROCESSING,b65835e3-b908-45c6-9995-ec98d0caafe7,b33b1cc9-81d8-4833-803f-466411581a4d,5600f641-b180-41b1-8043-921e66107744,None
1,695c1c55-bf6a-4265-b68d-4655999a7b85,2022-02-11T16:19:41.358733,2022-02-11T16:19:41.861241,DONE,08cc75ca-8d1e-4ead-9b56-ad1b6ba83a41,539c5e5e-25fa-457d-a2c0-ecf3f88c36f3,f158c26e-a94e-4994-9a95-de324aa1da23,None
2,9a71f775-c341-42b1-ab7a-cd0917b5e80f,2022-02-11T16:19:42.607900,2022-02-11T16:19:43.118409,DONE,08cc75ca-8d1e-4ead-9b56-ad1b6ba83a41,3275cec5-919c-4f18-bb4c-2ec941f7db3b,f158c26e-a94e-4994-9a95-de324aa1da23,None
3,8cf70e33-a879-4ab7-879f-5284da51c300,2022-02-11T16:19:43.835325,2022-02-11T16:19:44.321553,DONE,08cc75ca-8d1e-4ead-9b56-ad1b6ba83a41,63bd1d6d-3e81-48ae-9535-e04dc6d9d32f,f158c26e-a94e-4994-9a95-de324aa1da23,None
4,512b328b-6a19-4f7d-9cc1-a136edc9f344,2022-02-11T16:19:44.945428,2022-02-11T16:19:45.454695,DONE,08cc75ca-8d1e-4ead-9b56-ad1b6ba83a41,a5e39076-8b93-40f8-93b0-8b5edc65ef53,f158c26e-a94e-4994-9a95-de324aa1da23,None
...,...,...,...,...,...,...,...,...
214,b414b396-c3ad-437f-946e-8c7e3cc0c328,2022-02-18T20:37:14.432821,2022-02-18T20:37:14.432821,AVAILABLE,None,4aa34cf2-3fc8-4a9c-9658-87c52c21d21c,None,None
215,5d1c5c57-9189-44c8-998f-ff7679ec3159,2022-02-18T20:37:16.198965,2022-02-18T20:37:16.198965,AVAILABLE,None,2e52d545-2522-41f1-a90b-b22809e1f4b0,None,None
216,7ed71312-09de-4e73-aaac-b678b55054f6,2022-02-18T20:37:18.146182,2022-02-18T20:37:18.146182,AVAILABLE,None,cabc11d1-67ea-485c-bd9f-71a3004c004a,None,None
217,becaa60f-c923-4ac0-a4e3-0c31e333a12a,2022-02-18T20:37:19.989069,2022-02-18T20:37:19.989069,AVAILABLE,None,bbec210f-bf95-4f0c-8cb6-323d88c1c52d,None,None


In [6]:
# # load molecules from the first bacth
# with open('first_batch_smiles_uoft_only.txt', 'r') as f:
#     smiles_ = f.readlines()
    

In [19]:
# smiles = []
# hids = []
# for line in smiles_:
#
#     split = line.split('\t')
#     smiles.append(split[0])
#     hids.append(split[1].strip())

In [21]:
# mols = [chem.MolFromSmiles(s) for s in smiles]

In [22]:
# img = chem.Draw.MolsToGridImage(
#     mols, molsPerRow=10, subImgSize=(400,400), returnPNG=False, legends=hids,
# )
# img.save('first_round_samples_uoft_only_landscape.png')